<a href="https://colab.research.google.com/github/Kanishkirthik/SpamCall/blob/main/Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BI-DIRECTIONAL LSTM OUTPUT**

In [1]:
import numpy as np
import pandas as pd

In [2]:
df =pd.read_csv('sample_data/fraud_call.csv')

In [3]:
df.columns=["Label","Text"]
df=df.drop_duplicates()
df=df.dropna()

In [4]:
print(df.head())

    Label                                               Text
0   fraud  Todays Vodafone numbers ending with 4882 are s...
1  normal               Please don't say like that. Hi hi hi
2  normal                                         Thank you!
3  normal  Oh that was a forwarded message. I thought you...
4  normal  Got it. Seventeen pounds for seven hundred ml ...


In [5]:
df['Label'] = df['Label'].map({'fraud': 1, 'normal': 0})

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
texts = df["Text"].astype(str).tolist()
labels = df["Label"].tolist()

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [9]:
sequences = tokenizer.texts_to_sequences(texts)

In [10]:
maxlen=max([len(seq) for seq in sequences])

In [11]:
X=pad_sequences(sequences,maxlen=maxlen,padding='post')

In [12]:
y = np.array(labels)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,GRU,Layer,Flatten

In [14]:
print(df['Label'].value_counts())

Label
0    5014
1     596
Name: count, dtype: int64


In [15]:
from tensorflow.keras.layers import Bidirectional

In [16]:
from tensorflow.keras import layers,models

In [22]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1,output_dim=128, input_shape=(maxlen,)),  # Fixed input shape
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


model.build(input_shape=(None, maxlen))


model.summary()




/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 189, 128)            │       1,152,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 189, 256)            │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 189, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,584,449 (6.04 MB)

 Trainable params: 1,584,449 (6.04 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
import tensorflow as tf

precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', precision, recall, F1Score()]
)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
xt, X_temp, yt, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [26]:
history = model.fit(
    xt, yt,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=10
)

Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - accuracy: 0.9124 - f1_score: 0.3702 - loss: 0.2744 - precision_2: 0.6263 - recall_2: 0.2734 - val_accuracy: 0.9893 - val_f1_score: 0.9500 - val_loss: 0.0479 - val_precision_2: 0.9500 - val_recall_2: 0.9500
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.9880 - f1_score: 0.9435 - loss: 0.0435 - precision_2: 0.9688 - recall_2: 0.9199 - val_accuracy: 0.9893 - val_f1_score: 0.9508 - val_loss: 0.0531 - val_precision_2: 0.9355 - val_recall_2: 0.9667
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.9984 - f1_score: 0.9924 - loss: 0.0066 - precision_2: 0.9968 - recall_2: 0.9881 - val_accuracy: 0.9875 - val_f1_score: 0.9402 - val_loss: 0.0496 - val_precision_2: 0.9649 - val_recall_2: 0.9167
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.9968 - f1_score: 0.9843 - loss: 0.0145 - precision_2: 0.9848 - recall_2: 0.9838 - val_accuracy: 0.9733 - val_f1_score: 0.8760 - val_loss: 0.0823 - val

In [27]:
results1=model.evaluate(X_val,y_val)

18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - accuracy: 0.9822 - f1_score: 0.8938 - loss: 0.0529 - precision_2: 0.8762 - recall_2: 0.9209


In [28]:
print(f" validation Loss: {results1[0]}")
print(f"validation  Accuracy: {results1[1]}")
print(f" validation Precision: {results1[2]}")
print(f"validation  Recall: {results1[3]}")

 validation Loss: 0.09498398005962372
validation  Accuracy: 0.976827085018158
 validation Precision: 0.8615384697914124
validation  Recall: 0.9333333373069763


In [29]:
results2=model.evaluate(X_test,y_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 0.9802 - f1_score: 0.9089 - loss: 0.1278 - precision_2: 0.8949 - recall_2: 0.9262


In [30]:
print(f"Test Loss: {results2[0]}")
print(f"Test Accuracy: {results2[1]}")
print(f"Test Precision: {results2[2]}")
print(f"Test Recall: {results2[3]}")

Test Loss: 0.1282765120267868
Test Accuracy: 0.9786096215248108
Test Precision: 0.8852459192276001
Test Recall: 0.9152542352676392


In [31]:
results3=model.evaluate(xt,yt)

141/141 ━━━━━━━━━━━━━━━━━━━━ 48s 344ms/step - accuracy: 0.9959 - f1_score: 0.9781 - loss: 0.0127 - precision_2: 0.9581 - recall_2: 1.0000


In [32]:
print(f"Train Loss: {results3[0]}")
print(f"Train Accuracy: {results3[1]}")
print(f"Train Precision: {results3[2]}")
print(f"Train Recall: {results3[3]}")

Train Loss: 0.010416529141366482
Train Accuracy: 0.9971033930778503
Train Precision: 0.9734693765640259
Train Recall: 1.0


In [33]:
def classfiy(text):
  sequence=tokenizer.texts_to_sequences([text])
  padded_sequence=pad_sequences(sequence,maxlen=maxlen,padding='post')
  prediction=model.predict(padded_sequence,verbose=0)[0][0]
  print(prediction)
  if prediction>0.5:
    return 'fraud'
  else:
    return 'normal'

In [34]:
model.save("model.h5")

In [35]:
  print(classfiy('Todays Vodafone numbers ending with 4882 are selected to a receive a £350 award. If your number matches call 09064019014 to receive your £350 award'))

0.99972683
fraud


In [36]:
print(classfiy("Oh that was a forwarded message. I thought you send that to me."))

1.152131e-05
normal


In [37]:
!pip install flask flask-ngrok flask-cors tensorflow numpy

In [38]:
from tensorflow.keras.models import load_model

modeln = load_model("model.h5")

In [39]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [51]:
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})
run_with_ngrok(app)

In [52]:
@app.route("/")
def home():
    return "Keras ML Model API is running!"


In [53]:
@app.route("/predict", methods=["POST"])
def classfiy():
  data = request.json ;
  text = data["text"]
  sequence=tokenizer.texts_to_sequences([text])
  padded_sequence=pad_sequences(sequence,maxlen=maxlen,padding='post')
  prediction=modeln.predict(padded_sequence,verbose=0)[0][0]
  print(prediction)
  if prediction>0.5:
    return 'fraud'
  else:
    return 'normal'

In [54]:
!nohup python3 -m flask run --host=0.0.0.0 --port=5000 &


nohup: appending output to 'nohup.out'


In [45]:
!pip install pyngrok


In [56]:
from pyngrok import ngrok


In [55]:
!ngrok authtoken 2sheSevVOgcQrpU7EmxnVevN0Oi_87V5yaUY4zaFAqCTLiaLJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [58]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e8c1-34-69-123-226.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 05:38:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 05:38:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 05:38:38] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 05:38:40] "POST /predict HTTP/1.1" 200 -


0.99972683


...

0.99889696


INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 08:20:24] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 08:20:27] "POST /predict HTTP/1.1" 200 -


0.9991201


INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 08:21:35] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2025 08:21:36] "POST /predict HTTP/1.1" 200 -


0.9975751


In [57]:
public_url = ngrok.connect(5000)
print(f"Your API is live at: {public_url}")

Your API is live at: NgrokTunnel: "https://e8c1-34-69-123-226.ngrok-free.app" -> "http://localhost:5000"
